In [94]:
import pandas as pd;
import matplotlib.pyplot as plt;

In [95]:
train = pd.read_csv('train.csv');
test = pd.read_csv('test.csv');

train['Source'] = 'train';
test['Source'] = 'test';

In [96]:
df = pd.concat([train,test],ignore_index=True,sort=False);

In [97]:
df.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train


In [98]:
df.apply(lambda x: sum(x.isnull()))

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
Source                          0
dtype: int64

In [99]:
df.apply(lambda x: len(x.unique()))

Item_Identifier               1559
Item_Weight                    416
Item_Fat_Content                 5
Item_Visibility              13006
Item_Type                       16
Item_MRP                      8052
Outlet_Identifier               10
Outlet_Establishment_Year        9
Outlet_Size                      4
Outlet_Location_Type             3
Outlet_Type                      4
Item_Outlet_Sales             3494
Source                           2
dtype: int64

In [100]:
categorial_column = [x for x in df.dtypes.index if df.dtypes[x]=='object' and x not in ['Item_Identifier','Outlet_Identifier','Source']]

In [101]:
for col in categorial_column:
    print(df[col].value_counts(),"\n\n");

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64 


Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64 


Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64 


Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64 


Supermarket Type1    9294
Grocery Store        1805
Supermarket Type3    1559
Supermarket Type2    1546
Name: Outlet_Type, dtype: int64 




In [102]:
weightforItemType = pd.pivot_table(df,values='Item_Weight',index='Item_Type');

In [103]:
df.Item_Weight.fillna(df.Item_Weight.mean(skipna=True),inplace=True);

In [104]:
temp = pd.pivot_table(df,values='Outlet_Size',index='Outlet_Type',aggfunc=pd.Series.mode)
print(temp);

                  Outlet_Size
Outlet_Type                  
Grocery Store           Small
Supermarket Type1       Small
Supermarket Type2      Medium
Supermarket Type3      Medium


In [105]:
df.loc[df.Outlet_Size.isnull(),'Outlet_Size'] = df.loc[df.Outlet_Size.isnull(),'Outlet_Type'].apply(lambda x : temp.loc[x]);

In [106]:
df.apply(lambda x: sum(x.isnull()))

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
Source                          0
dtype: int64

In [107]:
visibility = pd.pivot_table(df,values='Item_Visibility',index='Item_Identifier');

In [108]:
df.loc[df['Item_Visibility']==0,'Item_Visibility'] = df.loc[df['Item_Visibility']==0,'Item_Identifier'].apply(lambda x: visibility.loc[x]);

In [109]:
print('Number of 0 values after modification: %d'%len(df[df['Item_Visibility'] == 0]))

Number of 0 values after modification: 0


In [110]:
df['Item_Visibility_MeanRatio'] = df['Item_Visibility'].apply(lambda x : x/df.Item_Visibility.mean())

In [111]:
print (df['Item_Visibility_MeanRatio'].describe())

count    14204.000000
mean         1.000000
std          0.713355
min          0.051280
25%          0.446785
50%          0.820456
75%          1.390487
max          4.710841
Name: Item_Visibility_MeanRatio, dtype: float64


In [112]:
df['Item_Type_Combined'] = df['Item_Identifier'].apply(lambda x: x[0:2]);

In [113]:
df['Item_Type_Combined'] = df['Item_Type_Combined'].map({'FD':'Food','NC':'Non-Consumable','DR':'Drinks'});

In [114]:
df.Item_Type_Combined.value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [115]:
df['Outlet_Years'] = 2013- df['Outlet_Establishment_Year'];

In [116]:
df['Item_Fat_Content'] = df.Item_Fat_Content.replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})

In [117]:
df.Item_Fat_Content.value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [118]:
from sklearn.preprocessing import LabelEncoder

In [119]:
le = LabelEncoder()

In [120]:
df['Outlet'] = le.fit_transform(df.Outlet_Identifier);
le = LabelEncoder()

In [121]:
col = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']

In [122]:
for c in col:
    df[c]= le.fit_transform(df[c]);

In [123]:
df = pd.get_dummies(df, columns=col);

In [124]:
df.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

In [125]:
train = df.loc[df.Source=='train'];
test = df.loc[df.Source=='test'];

In [126]:
train.drop(['Source'],axis=1,inplace=True);
test.drop(['Source','Item_Outlet_Sales'],axis=1,inplace=True)

/home/jaydev/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [127]:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

In [129]:
target = 'Item_Outlet_Sales';
IDcol = ['Item_Identifier','Outlet_Identifier']
from sklearn import cross_validation,metrics
import numpy as np

In [144]:
def modelfit(alg,dtrain,dtest,predictors,target,IDcol,filename):
    alg.fit(dtrain[predictors],dtrain[target]);
    cv_score = cross_validation.cross_val_score(alg,dtrain[predictors],dtrain[target],cv=20,scoring='mean_squared_error');
    cv_score = np.sqrt(np.abs(cv_score));
    print('score:',cv_score.mean());
    dtest[target] = alg.predict(dtest[predictors]);
    IDcol.append(target);
    submission = pd.DataFrame({x:dtest[x] for x in IDcol});
    submission.to_csv(filename,index=False);

In [145]:
from sklearn.linear_model import LinearRegression, Ridge,Lasso
predictors = [x for x in train.columns if x not in [target]+IDcol]
alg1 = LinearRegression(normalize=True);
modelfit(alg1,train,test,predictors,target,IDcol,'alg1.csv')

score: 1128.8692015853605


/home/jaydev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
